In [1]:
%config Completer.use_jedi = False

# To get csv from xmls

In [2]:
from preProcessing.utility import get_object_csv_from_xmls
DIR = '/home/l-ashwin/Datasets/mask_nomask/dummy/'
get_object_csv_from_xmls(DIR).head(2)

,imgname,width,height,class,xmin,xmax,ymin,ymax
0,1-200102094256105_png.rf.d2ed0c139e4e35ee8889a...,739,647,mask,196,581,55,471
1,135e-huxwryw6451820_jpg.rf.7119d8f128c2493e334...,581,848,mask,47,519,14,662


# CSV to tfrecord

In [3]:
import os
import pandas as pd
import tensorflow.compat.v1 as tf

In [4]:
# helper functions from tensorflow tutorial
from preProcessing.generate_tfrecord import split, create_tf_example
from object_detection.utils import label_map_util

In [5]:
DATA_DIR = '/home/l-ashwin/Datasets/mask_nomask/'
path_to_class_map = '/home/l-ashwin/Datasets/mask_nomask/labelmap.pbtxt'

In [6]:
label_map = label_map_util.load_labelmap(path_to_class_map)
label_map_dict = label_map_util.get_label_map_dict(label_map)

In [7]:
def class_text_to_int(row_label):
    return label_map_dict[row_label]

In [8]:
def write_tfrecord(path_to_imgs, path_to_csv, path_to_ouput_record):
    
    examples = pd.read_csv(path_to_csv)
    grouped  = split(examples, group='filename')
    writer   = tf.python_io.TFRecordWriter(path_to_ouput_record)
    
    for group in grouped:
        tf_example = create_tf_example(group, path_to_imgs, class_text_to_int)
        writer.write(tf_example.SerializeToString())
    writer.close()
    
    print('Successfully created the TFRecord file: {}'.format(path_to_ouput_record))

In [9]:
for kind in ['train', 'valid', 'test']:
    
    imgs_path    = os.path.join(DATA_DIR, kind)
    csv_path     = os.path.join(DATA_DIR, kind, '_annotations.csv')
    ouput_record = os.path.join('./data', kind, f'{kind}.record')
    
    write_tfrecord(imgs_path, csv_path, ouput_record)

Successfully created the TFRecord file: ./data/train/train.record
Successfully created the TFRecord file: ./data/valid/valid.record
Successfully created the TFRecord file: ./data/test/test.record
